In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os
from bs4 import BeautifulSoup
import re

import nltk.corpus
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from numpy.linalg import norm
from sklearn.metrics.pairwise import linear_kernel

import time
import webbrowser 

from multiprocessing.pool import ThreadPool as Pool
from multiprocessing import *

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
os.chdir(r"C:\Users\swani\Python Projects\Case Bucket\Data")

In [3]:
data = pd.read_excel('CD_UserQueries_clean.xlsx')
length = int(len(data))
print('Data Length: %d' %(length))
print(data.head(5))

Data Length: 1918
                                      Search Term  \
0                             probe depth working   
1  using environment variables timing check tfile   
2                            simulation profiling   
3                            timescale overriding   
4                          conditionally stopping   

                    Search Term Cleaned  
0                      probe depth work  
1  use environ variabl time check tfile  
2                          simul profil  
3                      timescal overrid  
4                           condit stop  


In [7]:
a = input()
start = time.time()
a = a.lower()
a = a.replace('[^\w\s]','')
stop_words = stopwords.words('english')
a = [word for word in a.split() if word not in (stop_words)]
a = ' '.join(a)
lmtzr = WordNetLemmatizer()
lemmatized = [lmtzr.lemmatize(word) for word in word_tokenize(a)]
a = ' '.join(lemmatized)
b = a
stemmr = PorterStemmer()
stemmed = [stemmr.stem(word) for word in word_tokenize(a)]
a = ' '.join(stemmed)
print('')
print('Processed input: %s' %(b))

searchTerm = data['Search Term Cleaned']
searchTerm_list = searchTerm.tolist()
searchTerm_display = data['Search Term']
searchTerm_display_list = searchTerm_display.tolist()
searchTerm_list.append(a)

vectorizer = TfidfVectorizer().fit_transform(searchTerm_list)
input_vector = vectorizer[-1]
vectorizer = vectorizer[:-1]

cosine_similarities = linear_kernel(vectorizer[:], input_vector).flatten()

sort_cs = np.sort(cosine_similarities)[::-1]
top3_values = sort_cs[:3]
print(top3_values)
if (any(i >= 0.5 for i in top3_values) is True):
    pass
else:
    top3_values = [0,0,0]
print('')
print("Cosine Similarities: ",top3_values)

indices_list = []

if (all(i == 0 for i in top3_values) is True):
    indices_list = [0,0,0]
else:
    for value in top3_values:
        max_list = [i for i,val in enumerate(cosine_similarities) if val==value]
        print(max_list)
        if max_list[0] in indices_list:
            if max_list[1] in indices_list:
                indices_list.append(max_list[2])
            else:
                indices_list.append(max_list[1])
        else: 
            indices_list.append(max_list[0])

print('')
print("Indices of 3 maximum value: ",indices_list)

print('')
print('Output (Similar N-Grams): ')
op_suggestions = []
if (indices_list == [0,0,0]):
    print("No suggestions match the input query")
else:
    for i in indices_list:
        op_suggestions.append(searchTerm_display_list[i])
    print(op_suggestions)

end = time.time()
print('')
print('Time Taken to Respond: %d seconds' %(end-start))

timescale overriding

Processed input: timescale overriding
[1.         0.73647798 0.51963309]

Cosine Similarities:  [1.         0.73647798 0.51963309]
[3]
[200]
[1347]

Indices of 3 maximum value:  [3, 200, 1347]

Output (Similar N-Grams): 
['timescale overriding', 'timescale ps', 'override module']

Time Taken to Respond: 0 seconds
